In [ ]:

#  This file demonstrates,
#
#     .  A better approach to json.loads()
#        Look for: import ast
#
#     .  Manual Cypher CREATE loops
#


#  Setup stuff: Connectivity

In [1]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [2]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [3]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [4]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 9vF4dnQAtR6wxKqoRo6ijDqmz5ZkAFfEbqaEN2S3Bdok, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


#  Group: 01

In [5]:

import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


--


In [6]:
#  Simple test, write, simple data types ..

df_data = pd.DataFrame([
   [101, "Sameer"    , "Katana"                , "Bay"    , "my_vertice"],
   [102, "Justin"    , "Katana"                , "LA"     , "my_vertice"],
   [103, "Brian"     , "Katana"                , "Denver" , "my_vertice"],
   ], columns = ["col1", "col2", "col3", "col4", "col5"])

df_data.head(20)


,col1,col2,col3,col4,col5
0,101,Sameer,Katana,Bay,my_vertice
1,102,Justin,Katana,LA,my_vertice
2,103,Brian,Katana,Denver,my_vertice


In [7]:

for l_each in df_data.itertuples():
    
   l_query = """

      CREATE ( n: {4} {{ colA: {0} }} )                         //  Notice double curly braces
      SET n.colB = '{1}', n.colC = '{2}', n.colD = '{3}'
      
      """.format(l_each.col1, l_each.col2, l_each.col3, l_each.col4, l_each.col5)
         
   l_result0 = my_graph.query(l_query)
        
print("--")


          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]

--


In [8]:

l_result = my_graph.query("""

   MATCH (n)
   RETURN n
   
   """)

print(l_result)


          0/? [?op/s]

   n.internal_id      n.labels  n.colA  n.colB  n.colC  n.colD n.type
0              1  [my_vertice]     102  Justin  Katana      LA   node
1              0  [my_vertice]     101  Sameer  Katana     Bay   node
2              2  [my_vertice]     103   Brian  Katana  Denver   node


#  Reset from above 

In [11]:

#  Reset from above, delete and recreate the graph

#  DELETE GRAPH
#
for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      l_handle=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

print("--")


--


In [12]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, EpqwGTEnhAqcFqrY59E2PMErdgzgo8hTdXGw2xtS8uLS, 0>


#  Group: 02

In [ ]:

#  Write the more complex dataframe ..

df_data = pd.DataFrame([
   [101, 30, {"ab":1, "we":2, "as":3}, "String1", "my_vertice"],
   [102, 31, {"ab":4, "we":5, "as":6}, "String2", "my_vertice"],
   [103, 32, {"ab":7, "we":8, "as":9}, "String2", "my_vertice"],
   ], columns = ["col1", "col2", "col3", "col4", "col5"])

df_data.head(20)


In [ ]:
#  Prove that we can (parse) the dataframe above, versus say "col3" being a string ..

for l_each in df_data.itertuples():
   #  print(type(l_each.col3))                               #  <class 'dict'>
   print(l_each.col3["ab"])


In [ ]:

#  import contextlib

for l_each in df_data.itertuples():
   l_query = """

      CREATE ( n: {4} {{ col1: {0} }} )                         //  Notice double curly braces
      SET n.col2 = {1}, n.col3 = \"{2}\", n.col4 = '{3}'
      
      """.format(l_each.col1, l_each.col2, l_each.col3, l_each.col4, l_each.col5)
         
   l_result0 = my_graph.query(l_query)
        
print("--")


In [ ]:

l_result = my_graph.query("""

   MATCH (n)
   RETURN n.col3 AS col3
   """)

print(l_result)


In [ ]:

#  import json, json.loads(),  does not work here because the DataFrame returned single quotes

import ast

for l_each in l_result.itertuples():
   l_aaa = ast.literal_eval(l_each.col3)
   print(l_aaa["ab"])
    
print("--")


#  Reset from above 

In [9]:
#  Reset from above, delete and recreate the graph

#  DELETE GRAPH
#
for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      l_handle=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

print("--")


--


In [10]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 7NFtpqB4udnu5j8TvFKuKkGxXfBovVonR6b2sX6WQTnY, 0>


#  Group: 03

In [ ]:
#  Write the more complex dataframe ..

df_data = pd.DataFrame([
   [101, 30, [ 1, 2, 3 ], "String1", "my_vertice"],
   [102, 31, [ 4, 5, 6 ], "String2", "my_vertice"],
   [103, 32, [ 7, 8, 9 ], "String2", "my_vertice"],
   ], columns = ["col1", "col2", "col3", "col4", "col5"])

df_data.head(20)


In [ ]:

#  Prove that we can (parse) the dataframe above, versus say "col3" being a string ..
#
#  Recall arrays and similar index from zero ..

print(df_data.col3[1][2])


In [ ]:

import contextlib

for l_each in df.iterrows():
    
   l_colA = l_each[1][0]
   l_colB = l_each[1][1]
   l_colC = l_each[1][2]
   l_colD = l_each[1][3]
   l_colE = l_each[1][4]
    
   l_query = """

      CREATE ( n: {4} {{ colA: {0} }} )                         //  Notice double curly braces
      SET n.colB = {1}, n.colC = {2}, n.colD = '{3}'
      
      """.format(l_colA, l_colB, l_colC, l_colD, l_colE)
         
   l_result0 = my_graph.query(l_query)
        
print("--")




In [ ]:
l_result = my_graph.query("""

   MATCH (n)
   RETURN n.colC[1]
   
   """)

print(l_result)

In [ ]:

#  This one errors out ..
#

l_result = my_graph.query("""

   MATCH (n)
   RETURN n.colC{'ab'}
   
   """)

print(l_result)


In [ ]:
# ##################################################################

In [ ]:

#  Below here, just DF manipulation
#

# Create dataframe where you'll store the dictionary values
# ddf = pd.DataFrame(columns=['AB','WE','AS'])
# 
# Populate ddf dataframe
# for (i,r) in df.iterrows():
#     e = r['Col C']
#     ddf.loc[i] = [e['ab'], e['we'], e['as']]
# 
# Replace df with the output of concat(df, ddf)
# df = pd.concat([df, ddf], axis=1)

# New column order, also drops old Col C column
# df = df[['Col A', 'Col B', 'AB', 'WE', 'AS', 'Col D']]

